In [1]:
import pandas as pd
import sqlalchemy

In [2]:
# Настройка параметров подключения к вашей базе данных
connection_params = {
    'dbname': 'mimic',
    'user': 'postgres',
    'host': 'localhost',
    'port': '5432'
}

In [3]:
# SQL-запрос
query = """
SELECT 
  drg.subject_id,
  drg.hadm_id,
  drg.drg_code,
  drg.description,
  ferritin.valuenum AS ferritin,
  ferrum.valuenum AS ferrum,
  haemoglobin.valuenum AS haemoglobin,
  platelets.valuenum AS platelets
FROM 
  drgcodes drg
LEFT JOIN (
  SELECT
    ll.subject_id,
    CAST(ll.hadm_id AS integer),
    ll.itemid,
    ll.valuenum,
    ll.charttime
  FROM labevents ll
  INNER JOIN
    (SELECT subject_id, hadm_id, itemid, MIN(charttime) AS mindate
    FROM labevents
    GROUP BY subject_id, hadm_id, itemid) lg
    ON ll.subject_id = lg.subject_id
    AND ll.hadm_id = lg.hadm_id
    AND ll.itemid = lg.itemid
    AND ll.charttime = lg.mindate
  WHERE
    ll.itemid = 50924
  ) ferritin ON drg.subject_id = ferritin.subject_id AND drg.hadm_id = ferritin.hadm_id
LEFT JOIN (
  SELECT
    ll.subject_id,
    CAST(ll.hadm_id AS integer),
    ll.itemid,
    ll.valuenum,
    ll.charttime
  FROM labevents ll
  INNER JOIN
    (SELECT subject_id, hadm_id, itemid, MIN(charttime) AS mindate
    FROM labevents
    GROUP BY subject_id, hadm_id, itemid) lg
    ON ll.subject_id = lg.subject_id
    AND ll.hadm_id = lg.hadm_id
    AND ll.itemid = lg.itemid
    AND ll.charttime = lg.mindate
  WHERE
    ll.itemid = 50952
) ferrum ON drg.subject_id = ferrum.subject_id AND drg.hadm_id = ferrum.hadm_id
LEFT JOIN (
  SELECT
    ll.subject_id,
    CAST(ll.hadm_id AS integer),
    ll.itemid,
    ll.valuenum,
    ll.charttime
  FROM labevents ll
  INNER JOIN
    (SELECT subject_id, hadm_id, itemid, MIN(charttime) AS mindate
    FROM labevents
    GROUP BY subject_id, hadm_id, itemid) lg
    ON ll.subject_id = lg.subject_id
    AND ll.hadm_id = lg.hadm_id
    AND ll.itemid = lg.itemid
    AND ll.charttime = lg.mindate
  WHERE
    ll.itemid = 50811
) haemoglobin ON drg.subject_id = haemoglobin.subject_id AND drg.hadm_id = haemoglobin.hadm_id
LEFT JOIN (
  SELECT
    ll.subject_id,
    CAST(ll.hadm_id AS integer),
    ll.itemid,
    ll.valuenum,
    ll.charttime
  FROM labevents ll
  INNER JOIN
    (SELECT subject_id, hadm_id, itemid, MIN(charttime) AS mindate
    FROM labevents
    GROUP BY subject_id, hadm_id, itemid) lg
    ON ll.subject_id = lg.subject_id
    AND ll.hadm_id = lg.hadm_id
    AND ll.itemid = lg.itemid
    AND ll.charttime = lg.mindate
  WHERE
    ll.itemid = 51265
) platelets ON drg.subject_id = platelets.subject_id AND drg.hadm_id = platelets.hadm_id
WHERE 
  drg.drg_code IN (374, 375, 376)
ORDER BY 
  drg.subject_id, drg.hadm_id ;
"""

In [4]:
uri = f"postgresql://{connection_params['user']}@{connection_params['host']}:{connection_params['port']}/{connection_params['dbname']}"
engine = sqlalchemy.create_engine(uri)

In [ ]:
df = pd.read_sql(query, con=engine);

In [ ]:
df

In [ ]:
# Сохранение DataFrame в файл CSV
df.to_csv('selected_data_crc.csv', index=False)  # index=False исключает сохранение индекса